In [8]:
import requests

In [23]:
from itertools import permutations

# Lista de tokens com preços já obtidos pela API
tokens = [
    {"symbol": "USDC", "address": "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"},
    {"symbol": "AAVE", "address": "0xD6DF932A45C0f255f85145f286eA0b292B21C90B"},
    {"symbol": "WETH", "address": "0x11CD37bb86F65419713f30673A480EA33c826872"},
    {"symbol": "WBTC", "address": "0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6"},
    {"symbol": "WMATIC", "address": "0xC3Ec80343D2bae2F8E680FDADDe7C17E71E114ea"},
    {"symbol": "LINK", "address": "0x2F6F07CDcf3588944Bf4C42aC74ff24bF56e7590"},
]


In [40]:
# Função para consultar preços na API do SushiSwap
def get_sushi_prices():
    url = "https://api.sushi.com/price/v1/137"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Falha ao obter preços da API do SushiSwap.")

# Função para realizar arbitragem triangular com os preços do SushiSwap
def triangular_arbitrage(prices, investment):
    """
    Executa arbitragem triangular usando os preços retornados pela API do SushiSwap.
    :param prices: Dicionário com os preços em dólares.
    :param investment: Valor inicial para arbitragem (em USD).
    :return: Resultados das combinações lucrativas.
    """
    # Obter os 20 primeiros preços disponíveis
    token_list = list(prices.items())[:6]
    tokens = [{"symbol": token, "price": price} for token, price in token_list]

    # Gerar todas as combinações de 3 tokens
    token_combinations = permutations(tokens, 3)
    profitable_cycles = []

    for token_a, token_b, token_c in token_combinations:
        # print(f"\n[CICLO] Testando: {token_a['symbol']} → {token_b['symbol']} → {token_c['symbol']} → {token_a['symbol']}")
        try:
            # Preços dos tokens
            price_a = token_a["price"]
            price_b = token_b["price"]
            price_c = token_c["price"]

            # Passo 1: A → B
            token_b_received = investment / price_a * price_b
            print(f"  [ETAPA 1] {investment:.2f} {token_a['symbol']} → {token_b_received:.6f} {token_b['symbol']}")

            # Passo 2: B → C
            token_c_received = token_b_received / price_b * price_c
            print(f"  [ETAPA 2] {token_b_received:.6f} {token_b['symbol']} → {token_c_received:.6f} {token_c['symbol']}")

            # Passo 3: C → A
            token_a_final = token_c_received / price_c * price_a
            print(f"  [ETAPA 3] {token_c_received:.6f} {token_c['symbol']} → {token_a_final:.2f} {token_a['symbol']}")

            # Calcular lucro
            profit = token_a_final - investment
            if profit > 0.01:
                print(f"  [RESULTADO] Lucro: {profit:.6f} USD")
                profitable_cycles.append({
                    "cycle": f"{token_a['symbol']} → {token_b['symbol']} → {token_c['symbol']} → {token_a['symbol']}",
                    "profit": profit,
                    "final_value": token_a_final,
                })
            # else:
            #     print(f"  [RESULTADO] Sem lucro neste ciclo.")
        except Exception as e:
            print(f"[ERRO] Falha ao processar ciclo: {e}")

    return profitable_cycles



In [41]:

# Obter preços da API do SushiSwap
sushi_prices = get_sushi_prices()

# Executar arbitragem triangular
investment = 1000  # Investimento inicial em USDC
# while True:
profitable_cycles = triangular_arbitrage(sushi_prices, investment)

# Exibir resultados
if profitable_cycles:
    print("\n[RESULTADOS] Ciclos Lucrativos:")
    for cycle in profitable_cycles:
        print(f"Cycle: {cycle['cycle']}, Profit: {cycle['profit']:.2f} USDC, Final Value: {cycle['final_value']:.2f} USDC")
else:
    print("\n[RESULTADOS] Nenhum ciclo lucrativo encontrado.")


  [ETAPA 1] 1000.00 0xc2132D05D31c914a87C6611C10748AEb04B58e8F → 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174
  [ETAPA 2] 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174 → 1000.516480 0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359
  [ETAPA 3] 1000.516480 0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359 → 1000.00 0xc2132D05D31c914a87C6611C10748AEb04B58e8F
  [ETAPA 1] 1000.00 0xc2132D05D31c914a87C6611C10748AEb04B58e8F → 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174
  [ETAPA 2] 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174 → 3224981.985391 0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619
  [ETAPA 3] 3224981.985391 0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619 → 1000.00 0xc2132D05D31c914a87C6611C10748AEb04B58e8F
  [ETAPA 1] 1000.00 0xc2132D05D31c914a87C6611C10748AEb04B58e8F → 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174
  [ETAPA 2] 1000.487276 0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174 → 96570462.778885 0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6
  [ETAPA 3] 965704